In [25]:
import tkinter as tk

class ChessGame:
    def __init__(self, master):
        self.master = master
        self.master.title("Chess Game")

        self.board = [
            ["r", "n", "b", "q", "k", "b", "n", "r"],
            ["p"] * 8,
            [""] * 8,
            [""] * 8,
            [""] * 8,
            [""] * 8,
            ["P"] * 8,
            ["R", "N", "B", "Q", "K", "B", "N", "R"]
        ]

        self.selected_piece = None
        self.valid_moves = set()

        self.create_board_gui()

    def create_board_gui(self):
        self.blocos = [[0 for _ in range(8)] for _ in range(8)]

        for i in range(8):
            for j in range(8):
                color = "#f0d9b5" if (i + j) % 2 == 0 else "#b58863"
                self.blocos[i][j] = tk.Button(self.master, width=6, height=3, bg=color, relief="flat",
                                              command=lambda i=i, j=j: self.click_square(i, j))
                self.blocos[i][j].grid(row=i, column=j)
                self.update_piece_gui(i, j)

    def update_piece_gui(self, row, col):
        piece = self.board[row][col]
        text = self.piece_to_unicode(piece) if piece else ""
        self.blocos[row][col].config(text=text, font=("Helvetica", 16, "bold"))

        if (row, col) in self.valid_moves:
            self.blocos[row][col].config(bg="#a1e1d4")
        else:
            color = "#f0d9b5" if (row + col) % 2 == 0 else "#b58863"
            self.blocos[row][col].config(bg=color)

    def click_square(self, row, col):
        if self.selected_piece is None:
            piece = self.board[row][col]
            if piece != "":
                self.selected_piece = (row, col)
                self.valid_moves = self.get_valid_moves(row, col)
        else:
            dest_row, dest_col = row, col
            src_row, src_col = self.selected_piece

            if (dest_row, dest_col) in self.valid_moves:
                self.board[dest_row][dest_col] = self.board[src_row][src_col]
                self.board[src_row][src_col] = ""
                self.update_piece_gui(src_row, src_col)
                self.update_piece_gui(dest_row, dest_col)

            self.selected_piece = None
            self.valid_moves = set()

    def is_valid_move(self, src_row, src_col, dest_row, dest_col):
        piece = self.board[src_row][src_col]
        dest_piece = self.board[dest_row][dest_col]

        # Lógica de movimento para peão
        if piece.lower() == "p":
            return self.valid_pawn_move(src_row, src_col, dest_row, dest_col)

        # Lógica de movimento para torre
        if piece.lower() == "r":
            return self.valid_rook_move(src_row, src_col, dest_row, dest_col)

        # Lógica de movimento para cavalo
        if piece.lower() == "n":
            return self.valid_knight_move(src_row, src_col, dest_row, dest_col)

        # Lógica de movimento para bispo
        if piece.lower() == "b":
            return self.valid_bishop_move(src_row, src_col, dest_row, dest_col)

        # Lógica de movimento para rainha
        if piece.lower() == "q":
            return self.valid_queen_move(src_row, src_col, dest_row, dest_col)

        # Lógica de movimento para rei
        if piece.lower() == "k":
            return self.valid_king_move(src_row, src_col, dest_row, dest_col)

        return False

    def valid_pawn_move(self, src_row, src_col, dest_row, dest_col):
        direction = 1 if self.board[src_row][src_col].islower() else -1

        # Movimento básico para frente
        if src_col == dest_col and self.board[dest_row][dest_col] == "":
            if src_row + direction == dest_row:
                return True
            elif src_row + 2 * direction == dest_row and self.board[src_row + direction][dest_col] == "":
                # Movimento inicial de duas casas
                return True

        # Captura diagonal
        if abs(src_col - dest_col) == 1 and src_row + direction == dest_row:
            return dest_piece and dest_piece.isupper() if direction == 1 else dest_piece and dest_piece.islower()

        return False

    def valid_rook_move(self, src_row, src_col, dest_row, dest_col):
        # Movimento na mesma linha ou coluna
        return src_row == dest_row or src_col == dest_col and self.clear_path(src_row, src_col, dest_row, dest_col)

    def clear_path(self, src_row, src_col, dest_row, dest_col):
        # Verificar se não há peças no caminho entre src e dest
        if src_row == dest_row:
            for col in range(min(src_col, dest_col) + 1, max(src_col, dest_col)):
                if self.board[src_row][col] != "":
                    return False
        elif src_col == dest_col:
            for row in range(min(src_row, dest_row) + 1, max(src_row, dest_row)):
                if self.board[row][src_col] != "":
                    return False
        return True

    def valid_knight_move(self, src_row, src_col, dest_row, dest_col):
        # Movimento em L
        return (abs(dest_row - src_row) == 2 and abs(dest_col - src_col) == 1) or \
               (abs(dest_col - src_col) == 2 and abs(dest_row - src_row) == 1)

    def valid_bishop_move(self, src_row, src_col, dest_row, dest_col):
        # Movimento na diagonal
        return abs(dest_row - src_row) == abs(dest_col - src_col) and self.clear_path(src_row, src_col, dest_row, dest_col)

    def valid_queen_move(self, src_row, src_col, dest_row, dest_col):
        # Movimento como torre ou bispo
        return (src_row == dest_row or src_col == dest_col or
                abs(dest_row - src_row) == abs(dest_col - src_col)) and self.clear_path(src_row, src_col, dest_row, dest_col)

    def valid_king_move(self, src_row, src_col, dest_row, dest_col):
        # Movimento para qualquer direção (uma casa)
        return abs(dest_row - src_row) <= 1 and abs(dest_col - src_col) <= 1

    def get_valid_moves(self, row, col):
        piece = self.board[row][col]
        valid_moves = set()

        for i in range(8):
            for j in range(8):
                dest_row, dest_col = i, j
                if self.is_valid_move(row, col, dest_row, dest_col):
                    valid_moves.add((dest_row, dest_col))

        return valid_moves

    def piece_to_unicode(self, piece):
        piece_unicode = {
            "p": "♟", "r": "♜", "n": "♞", "b": "♝", "q": "♛", "k": "♚",
            "P": "♙", "R": "♖", "N": "♘", "B": "♗", "Q": "♕", "K": "♔"
        }
        return piece_unicode.get(piece, "")

if __name__ == "__main__":
    root = tk.Tk()
    game = ChessGame(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Micro\AlunoPython\MyAnaconda\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Micro\AppData\Local\Temp\ipykernel_7224\4009696883.py", line 31, in <lambda>
    command=lambda i=i, j=j: self.click_square(i, j))
                             ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Micro\AppData\Local\Temp\ipykernel_7224\4009696883.py", line 51, in click_square
    self.valid_moves = self.get_valid_moves(row, col)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Micro\AppData\Local\Temp\ipykernel_7224\4009696883.py", line 153, in get_valid_moves
    if self.is_valid_move(row, col, dest_row, dest_col):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Micro\AppData\Local\Temp\ipykernel_7224\4009696883.py", line 71, in is_valid_move
    return self.valid_pawn_move(src_row, src_col, des